In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb

import matplotlib.pyplot as plt

In [ ]:
all = pd.read_csv('../data/Lymphoma-P1.csv')

In [ ]:
all.head(10)

In [ ]:
for col_name in ['Start Date', 'Primary Completion Date', 'Completion Date', 'First Posted', 'Last Update Posted']:
    all[col_name] = all[col_name].apply(lambda x: pd.to_datetime(x))
all['Completion Time'] = all['Completion Date'] - all['Start Date']

In [ ]:
all.head(10)

In [ ]:
#all['Completion Time'].dt.days.astype(int)
# Remove invalid rows
invalid_row_ids = []
for i, val in enumerate(all['Completion Time']):
    if type(val) != pd._libs.tslibs.timedeltas.Timedelta:
        invalid_row_ids.append(i)

all = all.drop(invalid_row_ids)
all = all.drop(['Start Date', 'Primary Completion Date', 'Completion Date', 'First Posted', 'Last Update Posted'], axis=1)

for i, val in enumerate(all['Completion Time']):
    if type(val) != pd._libs.tslibs.timedeltas.Timedelta:
        print((i, val))
all['Completion Time'] = all['Completion Time'].dt.days.astype(int)


In [ ]:
all.head(10)

In [ ]:
# Transfor string to class
class_mapping = {}
#col_names = ['Study Results']
#for col_name in col_names:
#    all[col_name].value_counts().head(10).index.tolist()
measure_count = {}
invalid_row_ids = []
for i, measures in enumerate(all['Outcome Measures']):
    if type(measures) != str:
        invalid_row_ids.append(i)
        continue
    for measure in measures.split('|'):
        measure = measure.strip()
        measure = measure.lower()
        #for key in measure_count.keys():
        #    if measure[:10] == key[:10]:
        #        measure_count[key] += 1
        #        break
        if measure not in measure_count:
            measure_count[measure] = 1
        else:
            measure_count[measure] += 1
print(sorted(measure_count.items(), key=lambda x: x[1], reverse=True)[:10])
all = all.drop(invalid_row_ids)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all.drop(['Completion Time'],axis=1), all['Completion Time'], 
                                                    test_size=0.33, random_state=42,)

In [ ]:
X_train.head(2)
X_test.head(2)
y_train.head(2)
y_test.head(2)

In [ ]:
y_test

In [ ]:
gbm = lgb.LGBMRegressor(n_estimators=200,)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l2')